# Youtube Api V3 usage for channel mining

# Install 
### pip install --upgrade google-api-python-client
### pip install --upgrade google-auth google-auth-oauthlib google-auth-httplib2
### pip install --upgrade oauth2client

In [1]:
import os, sys
import pandas as pd
import datetime
from pprint import pprint

from apiclient.discovery import build
from apiclient.errors import HttpError
from oauth2client.tools import argparser

import devkey
DEVELOPER_KEY = devkey.api1
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

In [2]:
YT = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY)

In [3]:
from quilt.data.haradai1262 import YouTuber
df = YouTuber.channel_videos.UUUM_videos()

In [ ]:
def get_comments( vid_tmp ):
    comment_threads = YT.commentThreads().list(
        part="snippet,replies",videoId=vid_tmp,maxResults=100,
        order='relevance',textFormat='plainText' ).execute()
    return comment_threads['items']

from time import sleep
RETRY = 10

comment_info = {}
idx = 0
for vid in df['id'].values:
    print( idx )
    for ridx in range(RETRY):
        try: 
            comments = get_comments( vid )
            break
        except:
            print( 'retry ...', ridx )
            sleep(5)
            continue
    for com in comments:
        comment_info[idx] = com['snippet']
        idx += 1

In [7]:
import json

f = open( 'comment_info.json', 'w' )
json.dump( comment_info, f )
f.close()

In [1]:
import json

f = open( 'comment_info.json', 'r' )
comment_info = json.load( f )

In [ ]:
comment_values = [
'videoId',
'id',
'totalReplyCount',
'authorChannelId',
'authorDisplayName',
'authorProfileImageUrl',
'likeCount',
'publishedAt',
'textOriginal',
'canRate',
'viewerRating'
]

def get_rows_comment( comment_info_tmp ):
    rows = []
    for i, com in comment_info_tmp.items():
        if int(i) % 10000 == 1: print( i )
        try:
            row = [
                com['videoId'],
                com['topLevelComment']['id'],
                com['totalReplyCount'],
                com['topLevelComment']['snippet']['authorChannelId']['value'],
                com['topLevelComment']['snippet']['authorDisplayName'],
                com['topLevelComment']['snippet']['authorProfileImageUrl'],
                com['topLevelComment']['snippet']['likeCount'],
                com['topLevelComment']['snippet']['publishedAt'],
                com['topLevelComment']['snippet']['textOriginal'],
                com['topLevelComment']['snippet']['canRate'],
                com['topLevelComment']['snippet']['viewerRating']
            ]
            rows.append( row )
        except:
            print( com )
            
    return rows

rows = get_rows_comment( comment_info )

In [7]:
import pandas as pd 
df = pd.DataFrame( rows, columns=comment_values )

In [10]:
df.head()

,videoId,id,totalReplyCount,authorChannelId,authorDisplayName,authorProfileImageUrl,likeCount,publishedAt,textOriginal,canRate,viewerRating
0,R7V5d94XkGQ,Ugzs3ZBVaiSHm03r1fl4AaABAg,229,UCZf__ehlCEBPop-_sldpBUQ,HikakinTV,https://yt3.ggpht.com/-NFhw6-eus8Y/AAAAAAAAAAI...,5488,2018-06-30T04:02:44.000Z,恭ちゃんときりたんぽちゃん初コラボ！\n寿司ヤバすぎたぜ。。。,True,none
1,R7V5d94XkGQ,UgwhLHAe53id_F5WsrF4AaABAg,10,UCO8LubzOAVucF8-ViSW9fbQ,その心笑ってるね,https://yt3.ggpht.com/-7iOPLipmr48/AAAAAAAAAAI...,461,2018-06-30T06:49:21.000Z,恭ちゃん水泳動画出して欲しいwwwww,True,none
2,R7V5d94XkGQ,UgyJIf9OD3XhghQn1SR4AaABAg,3,UCXxXHMzLsKZpGMwnkBMFu4A,性処理係シャミレイ,https://yt3.ggpht.com/-VVzSkjXRN-E/AAAAAAAAAAI...,252,2018-07-01T20:27:28.000Z,酢飯見たとき、牛丼のつゆだくご飯かと思った。,True,none
3,R7V5d94XkGQ,Ugw85Z56_7sxYrvZBBB4AaABAg,4,UCg69VIcN42rh57gDCayUNAA,Lian love,https://yt3.ggpht.com/-Vh_Ixc95q3c/AAAAAAAAAAI...,15,2018-07-06T13:47:45.000Z,きょうちゃんそんな風に見えないのに性格が、、、\nやっぱ画面だけだとわからない部分ありますよね,True,none
4,R7V5d94XkGQ,UgxZ2Vp-onZFQRpsCtt4AaABAg,5,UCrXdQ9KW6t1BV7F9Fx7acGg,速水もとこ,https://yt3.ggpht.com/-cZWbH7XpXAM/AAAAAAAAAAI...,87,2018-07-04T07:26:54.000Z,ヒカキンさん美味しいもの沢山食べてるはずなのに高級店でも知ったかしないし普通に値段に驚いたり...,True,none


In [11]:
df.to_csv( 'comment.csv', index=False  )